In [1]:
!pip install fastFM==0.2.9

     |████████████████████████████████| 259 kB 24.9 MB/s 
  Created wheel for fastFM: filename=fastFM-0.2.9-cp37-cp37m-linux_x86_64.whl size=431863 sha256=95433225493b94a6a4658c635c2090d70d9992d9ac274b0b0478444adcc2def6
  Stored in directory: /root/.cache/pip/wheels/73/64/93/8ac211b2001b1befac04bb1579dcf7bdd583afef4fd41c1a39
Successfully built fastFM


In [2]:
# !pip list | grep fastFM

In [ ]:
# !apt-get install python-dev libopenblas-dev 
# !git clone --recursive https://github.com/ibayer/fastFM.git
# import os
# os.chdir("/content/fastFM/")
# !pip install -r ./requirements.txt
# !make  
# !PYTHON=python3 make 
# !pip install .

In [2]:
import os, numpy as np
path = "/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/working_folder"
os.chdir(path)

FileNotFoundError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

import os 

os.chdir("/content/drive/My Drive")


Mounted at /content/drive


In [8]:
cd working_folder

[Errno 2] No such file or directory: 'working_folder'
/content/drive/.shortcut-targets-by-id/1ueeDBBseM0n9dGeC-1IU3k_cqOFHwJlf/working_folder


In [9]:
import gzip
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy
from collections import defaultdict
from fastFM import als
from scipy.spatial import distance
import csv
import dateutil.parser
from datetime import timedelta
import pickle
import numpy

In [10]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [11]:
with open("Cat_price_filtered_review_train.pkl","rb") as fp:
     all_dicts = pickle.load(fp)

with open("Cat_price_filtered_review_test.pkl","rb") as fp:
     test_dicts = pickle.load(fp)

# with open("Sampled_by_review_count_train_state.pkl","rb") as fp:
#      train_dicts = pickle.load(fp)

# with open("Sampled_by_review_count_val_state.pkl","rb") as fp:
#      val_dicts = pickle.load(fp)

# with open("Sampled_by_review_count_test_state.pkl","rb") as fp:
#      test_dicts = pickle.load(fp)

In [12]:
all_dicts.extend(test_dicts)

In [13]:
len(all_dicts)

749747

In [14]:
users = set([x['gPlusUserId'] for x in all_dicts])
nUsers = len(users)
places = set([x['gPlusPlaceId'] for x in all_dicts])
nPlaces = len(places)

allRatings = []
ratingDict = {}
userRatings = defaultdict(list) #all ratings given by user u
placeRatings = defaultdict(list)
usersPerPlace = defaultdict(set) # Maps an item to the users who rated it
placesPerUser = defaultdict(set) # Maps a user to the items that they rated

for d in all_dicts:
  user = d['gPlusUserId']
  place = d['gPlusPlaceId']
  usersPerPlace[place].add(user)
  placesPerUser[user].add(place)
  r = int(d['rating'])
  d['rating'] = r
  ratingDict[(user,place)] = r


In [15]:
review_count_per_user = [(len(placesPerUser[user]),user) for user in placesPerUser]
#avg_review_count = sum(review_count_per_user)/len(review_count_per_user)

sorted_review_count = sorted(review_count_per_user,reverse=True)

#7294 is the index till where reviews per user is 10 


user_list_rv_10 = [sorted_review_count[i][1] for i in range(7295)]

In [16]:
sorted_review_count[7294]

(10, '100008779363045501008')

In [17]:
len(user_list_rv_10)
user_list_rv_10 = set(user_list_rv_10)

In [18]:
req_dicts = []
for d in all_dicts:
  if d['gPlusUserId'] in user_list_rv_10:
    req_dicts.append(d)

In [19]:
len(req_dicts)

142227

In [24]:
for d in filtered_places:
  t = d["address"][-1].split()
  if len(t) >1 and t[-2] in state_list:
    d['state'] = t[-2]
  else:
    d['state'] = ""

place_id_state = defaultdict(str)
for d in filtered_places:
  if d['gPlusPlaceId'] not in place_id_state:
    place_id_state[d['gPlusPlaceId']] = d['state']

for d in req_dicts:
  d['state'] = place_id_state[d['gPlusPlaceId']]

In [25]:
from random import shuffle

shuffle(req_dicts)

In [26]:
train_dicts = req_dicts[:94000]
val_dicts = req_dicts[94000:117000]
test_dicts = req_dicts[117000:]

In [ ]:
# val_dicts = all_dicts[600000:]
# train_dicts = all_dicts[:600000]

In [ ]:
# with open("Cat_price_filtered_review_test.pkl","wb") as fp:
#      pickle.dump(test_dicts,fp)

In [ ]:
# with open("Cat_price_filtered_review_train.pkl","wb") as fp:
#      pickle.dump(train_dicts,fp)

In [20]:
with open("Price_filtered_places.pkl","rb") as fp:
     filtered_places = pickle.load(fp)

In [18]:
#Places mapping from place id to its attributes 

places_meta_data = defaultdict(dict)
for d in filtered_places:
  places_meta_data[d['gPlusPlaceId']] = d

In [26]:
train_dicts[0]

{'categories': ['Italian Restaurant', 'European Restaurant'],
 'gPlusPlaceId': '101700863372714982735',
 'gPlusUserId': '101970538350487280793',
 'gps': (39.285973, -76.601916),
 'rating': 5,
 'reviewText': None,
 'reviewTime': 'Jan 29, 2006',
 'reviewerName': 'robert cecil',
 'unixReviewTime': 1138579200}

In [27]:
# users = set([x['gPlusUserId'] for x in train_dicts])
# nUsers = len(users)
# places = set([x['gPlusPlaceId'] for x in train_dicts])
# nPlaces = len(places)

# prices = set([x['price'] for x in filtered_places])
# nPrices = len(prices)

# user_di = {user:i for i,user in enumerate(list(users))}
# place_di = {place:i for i,place in enumerate(list(places))}
# price_di = {price:i for i,price in enumerate(list(prices))}

In [ ]:
# X = scipy.sparse.lil_matrix((len(all_dicts), nUsers + nPlaces + nPrices))

# for i in range(len(all_dicts)):
#     user = user_di[all_dicts[i]['gPlusUserId']]
#     item = place_di[all_dicts[i]['gPlusPlaceId']]
#     price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
#     X[i,user] = 1 # One-hot encoding of user
#     X[i,nUsers + item] = 1 # One-hot encoding of item
#     X[i,nUsers + nPlaces+price] = 1

# y = numpy.array([d['rating'] for d in all_dicts])

# # split = int(0.8*len(all_dicts))
# # X_train,y_train = X[:split],y[:split]
# # X_val,y_val = X[split:],y[split:]

In [ ]:
# X_test = scipy.sparse.lil_matrix((len(test_dicts), nUsers + nPlaces + nPrices))

# for i in range(len(test_dicts)):
#   user_id = test_dicts[i]['gPlusUserId']
  
#   place_id = test_dicts[i]['gPlusPlaceId']
  
#   if(user_id in user_di and  place_id in place_di):
    
#     user = user_di[user_id]
#     item = place_di[place_id]
   
#     price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

#     X_test[i,user] = 1 # One-hot encoding of user
#     X_test[i,nUsers + item] = 1 # One-hot encoding of item
#     X_test[i,nUsers + nPlaces+price] = 1
    
#   elif(user_id in user_di):
#     user = user_di[user_id]
#     X_test[i,user] = 1 # One-hot encoding of user
#   elif(place_id in place_di):
#     item = place_di[place_id]
#     price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
#     X_test[i,nUsers + item] = 1 # One-hot encoding of item
#     X_test[i,nUsers + nPlaces+price] = 1

# y_test = numpy.array([d['rating'] for d in test_dicts])

In [ ]:
# fm = als.FMRegression(n_iter=500, init_stdev=0.1, rank=3, l2_reg_w=20, l2_reg_V=50)

# fm.fit(X_train, y_train)

In [ ]:
# y_pred_train = fm.predict(X_train)
# MSE(y_pred_train, y_train)

# y_pred_val = fm.predict(X_val)
# MSE(y_pred_val, y_val)

# y_test_pred = fm.predict(X_test)

1.1778189713842855

In [28]:
sorted_review_count = sorted(review_count_per_user,reverse=True)

In [29]:
sum(sorted_review_count[:25000])

TypeError: ignored

In [30]:
# for i in sorted_review_count:
#   if i == 2:
#     break
print(sorted_review_count.index(1))

ValueError: ignored

In [ ]:
sum(sorted_review_count[:40000]) + 80000+ 40000

381686

In [27]:
#Sampling scheme 

users_with_non1_rcount = [user for user in placesPerUser if len(placesPerUser[user]) >=2 ]
users_with_1_rcount = [user for user in placesPerUser if len(placesPerUser[user]) ==1 ]
users_with_1_rcount_subset = [random.choice(users_with_1_rcount) for _ in range(40000)]
users_with_non1_rcount.extend(users_with_1_rcount_subset)
user_list_sampled = set(users_with_non1_rcount)
sampled_train_dict = [d for d in train_dicts if d['gPlusUserId'] in user_list_sampled]

print(len(user_list_sampled))
print(len(sampled_train_dict))

140421
94000


In [ ]:
# with open("Sampled_by_review_count_train.pkl","wb") as fp:
#      pickle.dump(sampled_train_dict,fp)

In [ ]:
# train_dicts = sampled_train_dict

In [28]:
time_li = [d['unixReviewTime'] for d in train_dicts]
max_time = max(time_li)

users = set([x['gPlusUserId'] for x in train_dicts])
nUsers = len(users)
places = set([x['gPlusPlaceId'] for x in train_dicts])
nPlaces = len(places)

prices = set([x['price'] for x in filtered_places])
nPrices = len(prices)

user_di = {user:i for i,user in enumerate(list(users))}
place_di = {place:i for i,place in enumerate(list(places))}
price_di = {price:i for i,price in enumerate(list(prices))}

In [29]:
mod_train_dicts = []
allRatings = []
ratingDict = {}
userRatings = defaultdict(list) #all ratings given by user u
placeRatings = defaultdict(list)
usersPerPlace = defaultdict(set) # Maps an item to the users who rated it
placesPerUser = defaultdict(set) # Maps a user to the items that they rated

for d in train_dicts:
  user = d['gPlusUserId']
  place = d['gPlusPlaceId']
  usersPerPlace[place].add(user)
  placesPerUser[user].add(place)
  r = int(d['rating'])
  d['rating'] = r
  ratingDict[(user,place)] = r
  d['parsed_time'] = d['unixReviewTime']/max_time
  # for key in recipe_meta_data[place]:
  #   d[key] = recipe_meta_data[place][key]
  allRatings.append(r)
  userRatings[user].append((r,d['parsed_time']))
  placeRatings[place].append((r,d['parsed_time']))
  mod_train_dicts.append(d)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
  userAverage[u] = sum([t[0] for t in userRatings[u]]) / len(userRatings[u])

In [30]:
review_count_per_user = [len(placesPerUser[user]) for user in placesPerUser]
avg_review_count = sum(review_count_per_user)/len(review_count_per_user)
avg_review_count

12.885538039753255

In [31]:
y_val_glob_avg = [globalAverage]*len(y_val)

NameError: ignored

In [ ]:
MSE(y_val,y_val_glob_avg)

1.46733254963012

Vanilla FM

In [ ]:
X_train = scipy.sparse.lil_matrix((len(train_dicts), nUsers + nPlaces))

for i in range(len(train_dicts)):
    user = user_di[train_dicts[i]['gPlusUserId']]
    item = place_di[train_dicts[i]['gPlusPlaceId']]
    #price = price_di[places_meta_data[train_dicts[i]['gPlusPlaceId']]['price']]
    X_train[i,user] = 1 # One-hot encoding of user
    X_train[i,nUsers + item] = 1 # One-hot encoding of item
    #X_train[i,nUsers + nPlaces+price] = 1

y_train = numpy.array([d['rating'] for d in train_dicts])

# split = int(0.8*len(all_dicts))
# X_train,y_train = X[:split],y[:split]
# X_val,y_val = X[split:],y[split:]

In [ ]:
X_val = scipy.sparse.lil_matrix((len(val_dicts), nUsers + nPlaces))

for i in range(len(val_dicts)):
  user_id = val_dicts[i]['gPlusUserId']
  
  place_id = val_dicts[i]['gPlusPlaceId']
  
  if(user_id in user_di and  place_id in place_di):
    
    user = user_di[user_id]
    item = place_di[place_id]
   
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_val[i,user] = 1 # One-hot encoding of user
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_val[i,nUsers + nPlaces+price] = 1
    
  elif(user_id in user_di):
    user = user_di[user_id]
    X_val[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_val[i,nUsers + nPlaces+price] = 1

y_val = numpy.array([d['rating'] for d in val_dicts])

In [ ]:
X_test = scipy.sparse.lil_matrix((len(test_dicts), nUsers + nPlaces))

for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']
  
  place_id = test_dicts[i]['gPlusPlaceId']
  
  if(user_id in user_di and  place_id in place_di):
    
    user = user_di[user_id]
    item = place_di[place_id]
   
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_test[i,user] = 1 # One-hot encoding of user
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1
    
  elif(user_id in user_di):
    user = user_di[user_id]
    X_test[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

y_test = numpy.array([d['rating'] for d in test_dicts])

In [ ]:
fm = als.FMRegression(init_stdev=0.1, rank=0, l2_reg_w=20)

fm.fit(X_train, y_train)

FMRegression(l2_reg_w=20, rank=0)

In [ ]:
y_pred_train = fm.predict(X_train)
print(MSE(y_pred_train, y_train))

y_pred_val = fm.predict(X_val)
print(MSE(y_pred_val, y_val))

y_test_pred = fm.predict(X_test)

1.0264648031272718
1.4136195046086115


FastFM with state feature

In [16]:
state_list = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",""]

state_coords = [(32.806671,-86.79113),	(61.370716,-152.404419),	(33.729759,-111.431221),	(34.969704,-92.373123),	(36.116203,-119.681564),	(39.059811,-105.311104),	(41.597782,-72.755371),	(39.318523,-75.507141),	(27.766279,-81.686783),	(33.040619,-83.643074),	(21.094318,-157.498337),	(44.240459,-114.478828),	(40.349457,-88.986137),	(39.849426,-86.258278),	(42.011539,-93.210526),	(38.5266,-96.726486),	(37.66814,-84.670067),	(31.169546,-91.867805),	(44.693947,-69.381927),	(39.063946,-76.802101),	(42.230171,-71.530106),	(43.326618,-84.536095),	(45.694454,-93.900192),	(32.741646,-89.678696),	(38.456085,-92.288368),	(46.921925,-110.454353),	(41.12537,-98.268082),	(38.313515,-117.055374),	(43.452492,-71.563896),	(40.298904,-74.521011),	(34.840515,-106.248482),	(42.165726,-74.948051),	(35.630066,-79.806419),	(47.528912,-99.784012),	(40.388783,-82.764915),	(35.565342,-96.928917),	(44.572021,-122.070938),	(40.590752,-77.209755),	(41.680893,-71.51178),	(33.856892,-80.945007),	(44.299782,-99.438828),	(35.747845,-86.692345),	(31.054487,-97.563461),	(40.150032,-111.862434),	(44.045876,-72.710686),	(37.769337,-78.169968),	(47.400902,-121.490494),	(38.491226,-80.954453),	(44.268543,-89.616508),	(42.755966,-107.30249), (0.0,0.0)]

In [ ]:
nStates = 51
X_train = scipy.sparse.lil_matrix((len(train_dicts), nUsers + nPlaces+nStates))

for i in range(len(train_dicts)):
    user = user_di[train_dicts[i]['gPlusUserId']]
    item = place_di[train_dicts[i]['gPlusPlaceId']]
    state = state_list.index(train_dicts[i]['state'])
    #price = price_di[places_meta_data[train_dicts[i]['gPlusPlaceId']]['price']]
    X_train[i,user] = 1 # One-hot encoding of user
    X_train[i,nUsers + item] = 1 # One-hot encoding of item
    X_train[i,nUsers + nPlaces + state] = 1
    #X_train[i,nUsers + nPlaces+price] = 1

y_train = numpy.array([d['rating'] for d in train_dicts])

# split = int(0.8*len(all_dicts))
# X_train,y_train = X[:split],y[:split]
# X_val,y_val = X[split:],y[split:]

In [ ]:
# temp = X_train[0].todense()

In [ ]:
X_val = scipy.sparse.lil_matrix((len(val_dicts), nUsers + nPlaces+nStates))

for i in range(len(val_dicts)):
  user_id = val_dicts[i]['gPlusUserId']
  
  place_id = val_dicts[i]['gPlusPlaceId']
  state = state_list.index(val_dicts[i]['state'])
  X_val[i,nUsers + nPlaces + state] = 1
  if(user_id in user_di and  place_id in place_di):
    
    user = user_di[user_id]
    item = place_di[place_id]
    state = state_list.index(train_dicts[i]['state'])
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_val[i,user] = 1 # One-hot encoding of user
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_train[i,nUsers + nPlaces + state] = 1
    #X_val[i,nUsers + nPlaces+price] = 1
    
  elif(user_id in user_di):
    user = user_di[user_id]
    X_val[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_val[i,nUsers + nPlaces+price] = 1

y_val = numpy.array([d['rating'] for d in val_dicts])

In [ ]:
X_test = scipy.sparse.lil_matrix((len(test_dicts), nUsers + nPlaces+nStates))

for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']
  
  place_id = test_dicts[i]['gPlusPlaceId']
  state = state_list.index(test_dicts[i]['state'])
  X_test[i,nUsers + nPlaces + state] = 1
  if(user_id in user_di and  place_id in place_di):
    
    user = user_di[user_id]
    item = place_di[place_id]
   
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_test[i,user] = 1 # One-hot encoding of user
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1
    
  elif(user_id in user_di):
    user = user_di[user_id]
    X_test[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

y_test = numpy.array([d['rating'] for d in test_dicts])

In [ ]:
fm1 = als.FMRegression(init_stdev=0.1, rank=0, l2_reg_w=20)

fm1.fit(X_train, y_train)

FMRegression(l2_reg_w=20, rank=0)

In [ ]:
y_pred_train = fm1.predict(X_train)
print(MSE(y_pred_train, y_train))

y_pred_val = fm1.predict(X_val)
print(MSE(y_pred_val, y_val))

y_test_pred = fm1.predict(X_test)

1.025099194760506
1.4119281713148428


FISM with just places

In [32]:
X_train_fism = scipy.sparse.lil_matrix((len(train_dicts), nPlaces*2))

for i in range(len(train_dicts)):
    #user = user_di[train_dicts[i]['gPlusUserId']]
    user_id = train_dicts[i]['gPlusUserId']
    item = place_di[train_dicts[i]['gPlusPlaceId']]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_train_fism[i,item] = 1 # One-hot encoding of user
    for item_id in placesPerUser[user_id]:
      item_query = place_di[item_id]
      if item != item_query:
        X_train_fism[i,nPlaces + item_query] = 1/(len(placesPerUser[user_id]) - 1) # One-hot encoding of item
      
    #X_train_fism[i,nUsers + nPlaces+price] = 1

y_train_fism = numpy.array([d['rating'] for d in train_dicts])

In [29]:
X_val_fism = scipy.sparse.lil_matrix((len(val_dicts), nPlaces*2))

for i in range(len(val_dicts)):
  user_id = val_dicts[i]['gPlusUserId']
  
  place_id = val_dicts[i]['gPlusPlaceId']
  
  if(user_id in user_di and  place_id in place_di):
    user_id = val_dicts[i]['gPlusUserId']
    item = place_di[val_dicts[i]['gPlusPlaceId']]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val_fism[i,item] = 1 # One-hot encoding of user
    for item_id in placesPerUser[user_id]:
      item_query = place_di[item_id]
      if item != item_query:
        X_val_fism[i,nPlaces + item_query] = 1/(len(placesPerUser[user_id]) - 1)
    
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    #X_val_fism[i,nUsers + nPlaces+price] = 1
    
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val_fism[i,item] = 1 # One-hot encoding of item
    #X_val_fism[i,nUsers + nPlaces+price] = 1

y_val_fism = numpy.array([d['rating'] for d in val_dicts])

In [30]:
X_test_fism = scipy.sparse.lil_matrix((len(test_dicts), nPlaces*2))

for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']
  
  place_id = test_dicts[i]['gPlusPlaceId']
  
  if(user_id in user_di and  place_id in place_di):
    user_id = test_dicts[i]['gPlusUserId']
    item = place_di[test_dicts[i]['gPlusPlaceId']]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test_fism[i,item] = 1 # One-hot encoding of user
    for item_id in placesPerUser[user_id]:
      item_query = place_di[item_id]
      if item != item_query:
        X_test_fism[i,nPlaces + item_query] = 1/(len(placesPerUser[user_id]) - 1)
    
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    #X_val_fism[i,nUsers + nPlaces+price] = 1
    
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test_fism[i,item] = 1 # One-hot encoding of item
    #X_val_fism[i,nUsers + nPlaces+price] = 1

y_test_fism = numpy.array([d['rating'] for d in test_dicts])

In [31]:
fm_fism = als.FMRegression(init_stdev=0.1, rank=5, l2_reg_w=20, l2_reg_V=10)

fm_fism.fit(X_train_fism, y_train_fism)

y_pred_train = fm_fism.predict(X_train_fism)
print(MSE(y_pred_train, y_train_fism))

y_pred_val = fm_fism.predict(X_val_fism)
print(MSE(y_pred_val, y_val_fism))

y_test_pred = fm_fism.predict(X_test_fism)

0.8388888163085565
0.931783534839296


FISm with state co-ords 

In [ ]:

X_train = scipy.sparse.lil_matrix((len(train_dicts), nUsers + nPlaces+2))

for i in range(len(train_dicts)):
    user = user_di[train_dicts[i]['gPlusUserId']]
    item = place_di[train_dicts[i]['gPlusPlaceId']]
    state = state_list.index(train_dicts[i]['state'])
    cord = state_coords[state]
    #price = price_di[places_meta_data[train_dicts[i]['gPlusPlaceId']]['price']]
    X_train[i,user] = 1 # One-hot encoding of user
    X_train[i,nUsers + item] = 1 # One-hot encoding of item
    X_train[i,nUsers + nPlaces + 0] = cord[0]
    X_train[i,nUsers + nPlaces + 1] = cord[1]
    #X_train[i,nUsers + nPlaces+price] = 1

y_train = numpy.array([d['rating'] for d in train_dicts])

In [ ]:
X_val = scipy.sparse.lil_matrix((len(val_dicts), nUsers + nPlaces+2))

for i in range(len(val_dicts)):
  user_id = val_dicts[i]['gPlusUserId']
  
  place_id = val_dicts[i]['gPlusPlaceId']
  state = state_list.index(val_dicts[i]['state'])
  cord = state_coords[state]
  X_val[i,nUsers + nPlaces + 0] = cord[0]
  X_val[i,nUsers + nPlaces + 1] = cord[1]
  if(user_id in user_di and  place_id in place_di):
    
    user = user_di[user_id]
    item = place_di[place_id]
    state = state_list.index(train_dicts[i]['state'])
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_val[i,user] = 1 # One-hot encoding of user
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_train[i,nUsers + nPlaces + state] = 1
    #X_val[i,nUsers + nPlaces+price] = 1
    
  elif(user_id in user_di):
    user = user_di[user_id]
    X_val[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_val[i,nUsers + nPlaces+price] = 1

y_val = numpy.array([d['rating'] for d in val_dicts])

In [ ]:
X_test = scipy.sparse.lil_matrix((len(test_dicts), nUsers + nPlaces+2))

for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']
  
  place_id = test_dicts[i]['gPlusPlaceId']
  state = state_list.index(test_dicts[i]['state'])
  cord = state_coords[state]
  X_test[i,nUsers + nPlaces + 0] = cord[0]
  X_test[i,nUsers + nPlaces + 1] = cord[1]
  if(user_id in user_di and  place_id in place_di):
    
    user = user_di[user_id]
    item = place_di[place_id]
   
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_test[i,user] = 1 # One-hot encoding of user
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1
    
  elif(user_id in user_di):
    user = user_di[user_id]
    X_test[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

y_test = numpy.array([d['rating'] for d in test_dicts])

In [ ]:
fm2 = als.FMRegression(init_stdev=0.1, rank=0, l2_reg_w=20)

fm2.fit(X_train, y_train)

y_pred_train = fm2.predict(X_train)
print(MSE(y_pred_train, y_train))

y_pred_val = fm2.predict(X_val)
print(MSE(y_pred_val, y_val))

y_test_pred = fm2.predict(X_test)

1.0282877218789226
1.4168570405719452


#Fast FM with popularity of place and state

In [33]:
state_list = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",""]

state_coords = [(32.806671,-86.79113),	(61.370716,-152.404419),	(33.729759,-111.431221),	(34.969704,-92.373123),	(36.116203,-119.681564),	(39.059811,-105.311104),	(41.597782,-72.755371),	(39.318523,-75.507141),	(27.766279,-81.686783),	(33.040619,-83.643074),	(21.094318,-157.498337),	(44.240459,-114.478828),	(40.349457,-88.986137),	(39.849426,-86.258278),	(42.011539,-93.210526),	(38.5266,-96.726486),	(37.66814,-84.670067),	(31.169546,-91.867805),	(44.693947,-69.381927),	(39.063946,-76.802101),	(42.230171,-71.530106),	(43.326618,-84.536095),	(45.694454,-93.900192),	(32.741646,-89.678696),	(38.456085,-92.288368),	(46.921925,-110.454353),	(41.12537,-98.268082),	(38.313515,-117.055374),	(43.452492,-71.563896),	(40.298904,-74.521011),	(34.840515,-106.248482),	(42.165726,-74.948051),	(35.630066,-79.806419),	(47.528912,-99.784012),	(40.388783,-82.764915),	(35.565342,-96.928917),	(44.572021,-122.070938),	(40.590752,-77.209755),	(41.680893,-71.51178),	(33.856892,-80.945007),	(44.299782,-99.438828),	(35.747845,-86.692345),	(31.054487,-97.563461),	(40.150032,-111.862434),	(44.045876,-72.710686),	(37.769337,-78.169968),	(47.400902,-121.490494),	(38.491226,-80.954453),	(44.268543,-89.616508),	(42.755966,-107.30249), (0.0,0.0)]

In [34]:
placesPerState = defaultdict(int)

for d in train_dicts:
  placesPerState[d['state']] += 1

placesPerState[""] = 0

In [35]:

X_train = scipy.sparse.lil_matrix((len(train_dicts), nUsers + nPlaces+2))

for i in range(len(train_dicts)):
    user = user_di[train_dicts[i]['gPlusUserId']]
    item = place_di[train_dicts[i]['gPlusPlaceId']]
    state = state_list.index(train_dicts[i]['state'])
    cord = state_coords[state]
    #price = price_di[places_meta_data[train_dicts[i]['gPlusPlaceId']]['price']]
    X_train[i,user] = 1 # One-hot encoding of user
    X_train[i,nUsers + item] = 1 # One-hot encoding of item
    X_train[i,nUsers + nPlaces + 0] = len(usersPerPlace[train_dicts[i]['gPlusPlaceId']])
    X_train[i,nUsers + nPlaces + 1] = placesPerState[train_dicts[i]['state']]
    #X_train[i,nUsers + nPlaces+price] = 1

y_train = np.array([d['rating'] for d in train_dicts])




In [36]:
X_val = scipy.sparse.lil_matrix((len(val_dicts), nUsers + nPlaces+2))

for i in range(len(val_dicts)):
  user_id = val_dicts[i]['gPlusUserId']
  
  place_id = val_dicts[i]['gPlusPlaceId']
  state = state_list.index(val_dicts[i]['state'])
  cord = state_coords[state]
  X_val[i,nUsers + nPlaces + 0] = len(usersPerPlace[place_id])
  X_val[i,nUsers + nPlaces + 1] = placesPerState[val_dicts[i]['state']]
  if(user_id in user_di and  place_id in place_di):
    
    user = user_di[user_id]
    item = place_di[place_id]
    state = state_list.index(train_dicts[i]['state'])
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_val[i,user] = 1 # One-hot encoding of user
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_train[i,nUsers + nPlaces + state] = 1
    #X_val[i,nUsers + nPlaces+price] = 1
    
  elif(user_id in user_di):
    user = user_di[user_id]
    X_val[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_val[i,nUsers + nPlaces+price] = 1

y_val = np.array([d['rating'] for d in val_dicts])

In [37]:
X_test = scipy.sparse.lil_matrix((len(test_dicts), nUsers + nPlaces+2))

for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']
  
  place_id = test_dicts[i]['gPlusPlaceId']
  state = state_list.index(test_dicts[i]['state'])
  cord = state_coords[state]
  X_test[i,nUsers + nPlaces + 0] = len(usersPerPlace[place_id])
  X_test[i,nUsers + nPlaces + 1] = placesPerState[test_dicts[i]['state']]
  if(user_id in user_di and  place_id in place_di):
    
    user = user_di[user_id]
    item = place_di[place_id]
   
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_test[i,user] = 1 # One-hot encoding of user
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1
    
  elif(user_id in user_di):
    user = user_di[user_id]
    X_test[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

y_test = np.array([d['rating'] for d in test_dicts])

In [38]:
fm3 = als.FMRegression(init_stdev=0.1, rank=2, l2_reg_w=50)

fm3.fit(X_train, y_train)

y_pred_train = fm3.predict(X_train)
print(MSE(y_pred_train, y_train))

y_pred_val = fm3.predict(X_val)
print(MSE(y_pred_val, y_val))

y_test_pred = fm3.predict(X_test)

41.278912905009776
1926550.4665366483


In [39]:
print(MSE(y_test_pred, y_test))

2997711.895168805


Suprise

In [40]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 9.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619443 sha256=e4a5e937e1846a6a8c9b49ad85ad1796a5082ecdf96f7f29634ca4f98eec23e4
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [109]:
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split
from sklearn import linear_model
import sklearn
import gzip
import os
from collections import defaultdict
import random
import pandas as pd
from sklearn import linear_model
import dateutil.parser
from datetime import timedelta
from tqdm import tqdm
import csv
import pickle
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt

In [110]:
X_train = scipy.sparse.lil_matrix((len(train_dicts), nUsers + nPlaces+2))
X_train = []
for i in range(len(train_dicts)):
    user = train_dicts[i]['gPlusUserId']
    item = train_dicts[i]['gPlusPlaceId']
    state = state_list.index(train_dicts[i]['state'])
    cord = state_coords[state]
    #price = price_di[places_meta_data[train_dicts[i]['gPlusPlaceId']]['price']]

    X_train.append([user, item,  train_dicts[i]['rating']])
    #X_train[i,nUsers + nPlaces+price] = 1

y_train = np.array([d['rating'] for d in train_dicts])




In [111]:
#X_val = scipy.sparse.lil_matrix((len(val_dicts), nUsers + nPlaces+2))
X_val = []
for i in range(len(val_dicts)):
  user_id = val_dicts[i]['gPlusUserId']
  
  place_id = val_dicts[i]['gPlusPlaceId']

  res= []
  
  user = user_di[user_id]

  #item = place_di[place_id]
  #state = state_list.index(train_dicts[i]['state'])
  #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
  res.append(user_id)
  res.append(item_id)
  # One-hot encoding of item
  #X_train[i,nUsers + nPlaces + state] = 1
  #X_val[i,nUsers + nPlaces+price] = 1
  #res.append( len(usersPerPlace[place_id]))
  #res.append(placesPerState[val_dicts[i]['state']])
  res.append(val_dicts[i]['rating'])
  X_val.append(res)

  
y_val = np.array([d['rating'] for d in val_dicts])

In [112]:
training_data = X_train + X_val

In [113]:
df = pd.DataFrame(training_data, columns=['gPlusUserId', 'gPlusPlaceId','rating'])
df_train = df[['gPlusUserId', 'gPlusPlaceId', 'rating']]
df_train = df_train.head(int(0.8*len(training_data)))
train = df_train.values.tolist()
reader = Reader(rating_scale=(0, 5))
training_data = Dataset.load_from_df(df[['gPlusUserId', 'gPlusPlaceId', 'rating']], reader)

trainset, validationset = train_test_split(training_data, shuffle = False, test_size = 0.20)

In [121]:
model = SVD(lr_all= 0.005, n_factors= 1, reg_all= 15)    
model.fit(trainset)

In [122]:
predictions_train = model.test(train)
sse = 0
for p in predictions_train:
    sse += (p.r_ui - p.est)**2

print('Training set MSE is equal to ', sse / len(predictions_train))

Training set MSE is equal to  0.8759463336658917


In [146]:
predictions_val = model.test(validationset)

In [147]:
sse = 0
#predictions_val = [3.8480744680851062]*len(X_val)
for p in predictions_val:
    sse += (p.r_ui - p.est)**2
#sse = np.sum((predictions_val - y_val)**2)
print('Validation set MSE is equal to ', sse / len(predictions_val))

Validation set MSE is equal to  0.9506730957589691


In [125]:
#X_val = scipy.sparse.lil_matrix((len(val_dicts), nUsers + nPlaces+2))
X_test = []
for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']
  
  place_id = test_dicts[i]['gPlusPlaceId']

  res= []
  
  user = user_di[user_id]

  #item = place_di[place_id]
  #state = state_list.index(train_dicts[i]['state'])
  #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
  res.append(user_id)
  res.append(item_id)
  # One-hot encoding of item
  #X_train[i,nUsers + nPlaces + state] = 1
  #X_val[i,nUsers + nPlaces+price] = 1
  #res.append( len(usersPerPlace[place_id]))
  #res.append(placesPerState[val_dicts[i]['state']])
  res.append(test_dicts[i]['rating'])
  X_test.append(res)

  
y_test = np.array([d['rating'] for d in test_dicts])

In [149]:
predictions_test = model.test(X_test)
#predictions_test = [3.8480744680851062]*len(X_test)
sse = 0
for p in predictions_test:
  sse += (p.r_ui - p.est)**2
  #sse = np.sum((predictions_test - y_test)**2)

print('Test set MSE is equal to ', sse / len(predictions_test))

Test set MSE is equal to  0.9418578881499896


In [128]:
len(X_train)+len(X_val)+len(X_test)

142227

In [129]:
np.mean(y_train)

3.8480744680851062

In [138]:
y_test

array([4, 4, 4, ..., 5, 5, 4])

In [143]:
np.sum((predictions_test - y_test)**2)/len(X_test)

0.9578829290663099